# 1.1.2.2: Operaciones Matriciales y el Eje Computacional

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Realizar** operaciones matriciales elemento a elemento (suma, resta, escalar).
- **Calcular** el producto matriz-vector y matriz-matriz, interpretándolos como **transformaciones geométricas** de datos.
- **Visualizar** cómo una matriz puede rotar, escalar o sesgar un conjunto completo de puntos de datos a la vez.
- **Explicar** por qué la multiplicación de matrices en NumPy (usando `@`) es órdenes de magnitud más rápida que usar bucles.
- **Comprobar** y entender por qué la multiplicación de matrices **no es conmutativa** ($A \cdot B \neq B \cdot A$).

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

def plot_transformations(original_data, transformed_data, labels, title="Transformación de Datos"):
    """Función de ayuda para visualizar transformaciones de un conjunto de puntos."""
    plt.figure(figsize=(8, 8))
    # Datos originales en gris
    plt.scatter(original_data[:, 0], original_data[:, 1], c='gray', alpha=0.5, label='Original')
    
    colors = ['#E69F00', '#56B4E9', '#009E73', '#D55E00']
    # Datos transformados
    if isinstance(transformed_data, list):
        for i, data in enumerate(transformed_data):
            plt.scatter(data[:, 0], data[:, 1], c=colors[i % len(colors)], alpha=0.8, label=labels[i])
    else:
        plt.scatter(transformed_data[:, 0], transformed_data[:, 1], c=colors[0], alpha=0.8, label=labels)
        
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(True)
    plt.legend()
    plt.title(title)
    ax = plt.gca()
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)
    plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para este notebook, centrado en las transformaciones, los generadores de datos geométricos y de matrices especiales serán nuestros mejores aliados. Nos permitirán visualizar el efecto de cada operación de una manera muy intuitiva.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_business_data import create_business_data
from src.data_generation.create_geometric_shapes import create_geometric_shapes
from src.data_generation.create_special_matrices import create_special_matrices

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Datos a transformar)
# Nuestro dataset de estudiantes servirá como un conjunto de puntos del mundo real
# al que aplicaremos transformaciones, como "curvar" las notas o re-escalar las features.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=80)

# 💡 CONTEXTO PEDAGÓGICO: Visualización Geométrica Pura
# No hay mejor manera de entender una transformación que viéndola. Generaremos un círculo
# de puntos para ver claramente cómo la multiplicación matricial lo rota, estira o sesga.
datos_circulo = create_geometric_shapes(rng, shape_type='circle', n_samples=100, noise_level=0)

# 💡 CONTEXTO PEDAGÓGICO: Operaciones en Negocios
# Usaremos datos de negocio para contextualizar operaciones como sumar ventas de dos trimestres
# o aplicar una matriz de conversión de moneda a una matriz de precios.
ventas_q1 = create_business_data(rng, n_samples=10).add_prefix('prod_')
ventas_q2 = create_business_data(rng, n_samples=10).add_prefix('prod_')

print("Datasets generados y listos para usar.")

## 1. Operaciones Elemento a Elemento

La suma, resta y multiplicación por un escalar funcionan exactamente como con los vectores: se aplican a cada elemento de la matriz de forma independiente. Para la suma y la resta, las matrices deben tener **exactamente la misma dimensión**.

En NumPy, estas operaciones se realizan con los operadores `+`, `-` y `*`.

### Ejemplo Demostrativo 1: Suma de Matrices de Ventas

In [ ]:
# 1. DATOS: Dos matrices de ventas (simplificadas a 10x2).
matriz_ventas_q1 = ventas_q1[['prod_precio', 'prod_ventas_mensuales']].values
matriz_ventas_q2 = ventas_q2[['prod_precio', 'prod_ventas_mensuales']].values # Usamos el mismo precio para Q2

# 2. APLICACIÓN DEL CONCEPTO: Suma elemento a elemento para obtener ventas totales.
# Aquí, sumar precios no tiene sentido, pero sumar ventas sí. Es un ejemplo conceptual.
ventas_semestre = matriz_ventas_q1 + matriz_ventas_q2

# 3. INTERPRETACIÓN
print("Ventas Q1 (primeros 3 prods):\n", np.round(matriz_ventas_q1[:3], 2))
print("\nVentas Q2 (primeros 3 prods):\n", np.round(matriz_ventas_q2[:3], 2))
print("\nVentas Totales Semestre (primeros 3 prods):\n", np.round(ventas_semestre[:3], 2))
print(f"\nForma Q1: {matriz_ventas_q1.shape}, Forma Q2: {matriz_ventas_q2.shape} -> Forma Total: {ventas_semestre.shape}")
print("La suma mantiene las dimensiones.")

## 2. Producto Matriz-Vector y Matriz-Matriz: El Corazón del Álgebra Lineal

Esta es la operación más importante. A diferencia de las operaciones elemento a elemento, la multiplicación de matrices (con el operador `@` en NumPy) tiene un significado profundo:

- **Producto Matriz-Vector ($A \cdot \vec{v}$):** Es una **transformación lineal**. La matriz $A$ "actúa" sobre el vector $\vec{v}$ y lo convierte en un nuevo vector. Es la forma de rotar, escalar o sesgar un punto en el espacio.

- **Producto Matriz-Matriz ($A \cdot B$):** Es una **composición de transformaciones**. La matriz resultante $C = A \cdot B$ es una nueva transformación que equivale a aplicar primero la transformación $B$ y *luego* la transformación $A$.

### Regla de Dimensiones
Para que la multiplicación $A \cdot B$ sea posible, el número de columnas de $A$ debe ser igual al número de filas de $B$.

Si $A$ es $(m \times \textbf{n})$ y $B$ es $(\textbf{n} \times p)$, el resultado $C$ será $(m \times p)$.


### Ejemplo Demostrativo 2: Transformando un Conjunto de Puntos
Aquí está la magia. Podemos aplicar una transformación a miles de puntos a la vez con una sola operación.

In [ ]:
# 1. DATOS: Nuestro círculo de 100 puntos (matriz 100x2).
puntos_circulo = datos_circulo[['x', 'y']].values

# 2. DEFINIR TRANSFORMACIONES: Creamos varias matrices de transformación 2x2.
T_rotacion = np.array([[np.cos(np.pi/4), -np.sin(np.pi/4)], [np.sin(np.pi/4), np.cos(np.pi/4)]]) # Rotación 45°
T_escala = np.array([[2, 0], [0, 0.5]]) # Estirar en X, encoger en Y
T_cizalla = np.array([[1, 0.5], [0, 1]]) # Shear o cizalla horizontal

# 3. APLICAR TRANSFORMACIONES
# Para aplicar T (2x2) a P (100x2), debemos transponer P para que las dimensiones coincidan:
# (2x2) @ (2x100) -> (2x100). Luego transponemos el resultado de vuelta a (100x2).
puntos_rotados = (T_rotacion @ puntos_circulo.T).T
puntos_escalados = (T_escala @ puntos_circulo.T).T
puntos_cizallados = (T_cizalla @ puntos_circulo.T).T

# 4. VISUALIZACIÓN
plot_transformations(
    original_data=puntos_circulo,
    transformed_data=[puntos_rotados, puntos_escalados, puntos_cizallados],
    labels=['Rotado', 'Escalado', 'Cizallado'],
    title='Aplicando Diferentes Transformaciones a un Círculo'
)

### Ejemplo Demostrativo 3: La No Conmutatividad Importa
¿Es lo mismo rotar y luego estirar, que estirar y luego rotar? No. El orden de las multiplicaciones matriciales es crucial.

In [ ]:
# 1. DATOS: Usamos el mismo círculo y las mismas matrices de rotación y escala.
T_rotacion = np.array([[np.cos(np.pi/4), -np.sin(np.pi/4)], [np.sin(np.pi/4), np.cos(np.pi/4)]])
T_escala = np.array([[2, 0], [0, 0.5]])

# 2. APLICACIÓN: Componemos las transformaciones en órdenes opuestos.
Comp_RotarLuegoEscalar = T_escala @ T_rotacion
Comp_EscalarLuegoRotar = T_rotacion @ T_escala

# Aplicamos las transformaciones compuestas al círculo
resultado1 = (Comp_RotarLuegoEscalar @ puntos_circulo.T).T
resultado2 = (Comp_EscalarLuegoRotar @ puntos_circulo.T).T

# 3. VISUALIZACIÓN
print(f"Matriz Rotar->Escalar:\n{np.round(Comp_RotarLuegoEscalar, 2)}")
print(f"\nMatriz Escalar->Rotar:\n{np.round(Comp_EscalarLuegoRotar, 2)}")
plot_transformations(
    original_data=puntos_circulo,
    transformed_data=[resultado1, resultado2],
    labels=['Rotar -> Escalar', 'Escalar -> Rotar'],
    title='La Multiplicación de Matrices NO es Conmutativa'
)

### 🚀 El Eje Computacional: La Fuerza Bruta de la Multiplicación Vectorizada

La multiplicación de matrices es computacionalmente costosa ($O(n^3)$ con el algoritmo ingenuo). Una implementación en Python puro con bucles es extremadamente lenta. NumPy utiliza librerías de bajo nivel (BLAS, LAPACK) escritas en C y Fortran, que están altamente optimizadas para aprovechar el hardware moderno. La diferencia no es pequeña, es abismal.

In [ ]:
# Creamos dos matrices cuadradas de 300x300
A_np = rng.random((300, 300))
B_np = rng.random((300, 300))

# Método 1: Multiplicación vectorizada de NumPy con @
print("Midiendo el tiempo de la multiplicación vectorizada de NumPy:")
%timeit C_np = A_np @ B_np

# Método 2: Multiplicación con bucles de Python
def multiplica_con_bucles(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
    return C

print("\nMidiendo el tiempo de la multiplicación con bucles de Python (esto puede tardar...):")
%timeit C_loop = multiplica_con_bucles(A_np, B_np)

print("\n¡La diferencia de velocidad es de varios órdenes de magnitud! Siempre usa `@` para la multiplicación de matrices.")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Compatibilidad de Dimensiones ===

In [ ]:
A = rng.random((3, 4))
B = rng.random((4, 2))
C = rng.random((3, 2))

# TODO 1: Predice la forma del resultado de A @ B.
forma_esperada_AB = # COMPLETAR (escribe una tupla, ej: (filas, columnas))

resultado_AB = A @ B
assert resultado_AB.shape == forma_esperada_AB, f"La forma real {resultado_AB.shape} no coincide con la esperada {forma_esperada_AB}"
print(f"✅ A(3x4) @ B(4x2) -> C{forma_esperada_AB}. ¡Correcto!")

# TODO 2: ¿Es posible calcular A @ C? En un comentario, explica por qué sí o por qué no.
# Explicación: # COMPLETAR

### === EJERCICIO GUIADO 2: Aplicar una Transformación a un Vector ===

In [ ]:
# DATOS: Una matriz de transformación y un vector de estudiante.
T_reflexion_y = np.array([[-1, 0], [0, 1]]) # Refleja sobre el eje Y
v_estudiante = datos_estudiantes[['horas_estudio', 'calificacion_examen']].iloc[0].values

# TODO: Aplica la transformación T al vector v_estudiante.
v_transformado = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert v_transformado.shape == (2,), "El resultado debería ser un vector de 2 dimensiones."
assert np.isclose(v_transformado[0], -v_estudiante[0]), "El componente X debería haberse invertido."
print("✅ ¡Transformación correcta!")
print(f"Vector Original: {np.round(v_estudiante, 2)}")
print(f"Vector Transformado: {np.round(v_transformado, 2)}")

### === EJERCICIO GUIADO 3: Suma de Matrices de Datos ===

In [ ]:
# DATOS: Dos matrices 5x2 de datos de estudiantes de dos grupos diferentes.
grupo_A = datos_estudiantes[['horas_estudio', 'calificacion_examen']].head(5).values
grupo_B = datos_estudiantes[['horas_estudio', 'calificacion_examen']].tail(5).values

# TODO: Calcula una matriz 'promedio_grupos' que sea el promedio elemento a elemento de ambos grupos.
matriz_promedio = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert matriz_promedio.shape == (5, 2), "La forma de la matriz promedio es incorrecta."
assert np.allclose(matriz_promedio[0, 0], (grupo_A[0, 0] + grupo_B[0, 0]) / 2), "El cálculo del promedio es incorrecto."
print("✅ ¡Cálculo de matriz promedio correcto!")
print(np.round(matriz_promedio, 2))

### === EJERCICIO GUIADO 4: Aplicar Impuestos y Envío a Productos ===

In [ ]:
# DATOS: Una matriz 10x2 con [precio, peso] de 10 productos.
matriz_productos = create_business_data(rng, n_samples=10)[['precio', 'calificacion_cliente']].values # Usamos calificacion_cliente como 'peso' conceptual
matriz_productos[:, 1] *= 5 # Aumentamos el peso para que sea más visible

# TODO: Crea una matriz de transformación 2x2 que:
# - Aplique un 19% de IVA al precio (multiplicar por 1.19).
# - Calcule un costo de envío de 3 por unidad de peso.
# El resultado debe ser una matriz con [precio_final, costo_envio]
T_costos = # COMPLETAR. PISTA: [[1.19, 0], [0, 3]]

# TODO: Calcula la matriz de costos finales.
# ¡Cuidado con las dimensiones! Tienes (10x2) y (2x2).
matriz_costos_finales = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
assert matriz_costos_finales.shape == (10, 2), "La forma final es incorrecta."
assert np.allclose(matriz_costos_finales[0, 0], matriz_productos[0, 0] * 1.19), "El cálculo del IVA es incorrecto."
print("✅ ¡Cálculo de costos correcto!")

### === EJERCICIO GUIADO 5: Composición de Transformaciones ===

In [ ]:
# DATOS: Una matriz de rotación y una de cizalla.
T_rot = np.array([[0, -1], [1, 0]]) # Rotación de 90 grados
T_ciz = np.array([[1, 0.5], [0, 1]]) # Cizalla horizontal

# TODO: Crea una matriz 'T_compuesta' que represente aplicar PRIMERO la cizalla y LUEGO la rotación.
T_compuesta = # COMPLETAR. PISTA: El orden de multiplicación importa.

# VERIFICACIÓN AUTOMÁTICA
v = np.array([1, 0])
v_cizallado = T_ciz @ v
v_final_esperado = T_rot @ v_cizallado
v_final_compuesto = T_compuesta @ v
assert np.allclose(v_final_esperado, v_final_compuesto), "La matriz compuesta no produce el resultado esperado."
print("✅ ¡Matriz compuesta correcta!")
print(T_compuesta)

### === EJERCICIO GUIADO 6: Producto de Hadamard (Elemento a Elemento) vs. Producto Matricial ===

In [ ]:
# DATOS
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

# TODO 1: Calcula el producto de Hadamard (elemento a elemento).
# PISTA: En NumPy, esto se hace con el operador '*'.
Hadamard = # COMPLETAR

# TODO 2: Calcula el producto matricial estándar.
# PISTA: Usa el operador '@'.
Matricial = # COMPLETAR

# VERIFICACIÓN
assert Hadamard[0, 0] == 5 and Hadamard[1, 1] == 32
assert Matricial[0, 0] == 19 and Matricial[1, 1] == 50
print("✅ ¡Cálculos correctos!")
print(f"Producto de Hadamard (A * B):\n{Hadamard}")
print(f"\nProducto Matricial (A @ B):\n{Matricial}")
print("\nMoraleja: NUNCA confundas '*' con '@' para la multiplicación de matrices.")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Operaciones Básicas y Dimensiones

**A1 (🟢 Fácil):** Dadas `A = np.array([[1,2],[3,4]])` y `B = np.array([[10,0],[0,10]])`, calcula `A + B` y `10 * A`.

**A2 (🟢 Fácil):** Dadas `C = np.array([[1,2,3],[4,5,6]])` (2x3) y `D = np.array([[7,8],[9,10],[11,12]])` (3x2), calcula `C @ D`. ¿Cuál es la dimensión del resultado?

**A3 (🟢 Fácil):** Con las matrices `C` y `D` del ejercicio anterior, intenta calcular `C @ C`. ¿Por qué falla? Explícalo en un comentario.

**A4 (🟡 Medio):** Con las matrices `C` y `D` anteriores, calcula `D @ C`. ¿Es el resultado igual a `C @ D`? ¿Qué demuestra esto?

**A5 (🟡 Medio):** Crea una matriz 4x4 identidad, `I`. Crea una matriz aleatoria 4x4, `A`. Verifica que `A @ I` es igual a `I @ A` y que ambos son iguales a `A`.

**A6 (🔴 Reto):** Una matriz de datos `D` tiene 100 observaciones (filas) y 5 features (columnas). Quieres pre-multiplicarla por una matriz de pesos `W` para obtener una única puntuación por observación. ¿Qué dimensiones debe tener `W` para que el producto `D @ W` resulte en una matriz (vector columna) de 100x1?

### Parte B: Transformaciones Geométricas

**B1 (🟢 Fácil):** La matriz `S = np.array([[2,0],[0,0.5]])` estira el eje x al doble y encoge el eje y a la mitad. Aplica esta transformación al vector `u = np.array([3,4])` y muestra el resultado.

**B2 (🟢 Fácil):** La matriz `P_x = np.array([[1, 0], [0, 0]])` proyecta cualquier vector sobre el eje x. Aplícala al vector `u = np.array([3,4])`.

**B3 (🟡 Medio):** Genera un conjunto de 50 puntos en forma de `parabola` usando `create_geometric_shapes`. Aplica la matriz de reflexión sobre el eje x, `R_x = np.array([[1, 0], [0, -1]])`, a todos los puntos a la vez. Visualiza el resultado con `plot_transformations`.

**B4 (🟡 Medio):** Crea una matriz de rotación para -30 grados y aplícala al `datos_circulo`. Visualiza el resultado.

**B5 (🔴 Reto):** ¿Qué matriz obtienes si multiplicas la matriz de rotación de 90 grados por sí misma (`R @ R`)? ¿Qué transformación representa el resultado? Demuéstralo visualmente aplicando `R @ R` al `datos_circulo`.

**B6 (🔴 Reto):** Crea una matriz de transformación `T` 2x2 que realice una "cizalla" o *shear* vertical, transformando el vector `[0,1]` en `[0.5, 1]` y dejando el vector `[1,0]` sin cambios. Aplica esta transformación al `datos_estudiantes` y visualiza el resultado.

### Parte C: Aplicaciones con Datasets

**C1 (🟢 Fácil):** Toma los primeros 5x5 de la matriz `datos_estudiantes` y multiplícala por un escalar para "bonificar" a todos: suma 5 puntos a la calificación de examen.

**C2 (🟡 Medio):** Supón que tienes una matriz de `datos_estudiantes` (10x2) con `[horas, calif]`. Tienes un vector de 'pesos' `w = [0.4, 0.6]`. Calcula una puntuación final para cada estudiante usando `estudiantes @ w`. ¿Qué forma tiene el resultado y qué representa?

**C3 (🔴 Reto):** Tienes una matriz de precios en euros `P` de 20x3 (3 productos). Quieres calcular el precio final en USD y JPY. Crea una matriz de conversión `C` de 3x2, donde cada columna es un producto y cada fila es una moneda (EUR->USD, EUR->JPY). Calcula la matriz de precios finales `P_final = P @ C`. ¿Qué representan las columnas de la matriz final?

---

## ✅ Mini-Quiz de Autoevaluación

1. Para multiplicar una matriz 5x3 por una matriz B, ¿qué dimensión debe tener B?
2. ¿Cuál es la razón principal por la que la multiplicación de matrices en NumPy es mucho más rápida que implementarla con bucles de Python?
3. Si `A` es una rotación y `B` es un estiramiento, ¿por qué `A @ B` generalmente no es lo mismo que `B @ A`? (Piensa geométricamente).
4. ¿Qué operador se usa en NumPy para la multiplicación de matrices (producto matricial) y cuál para la multiplicación elemento a elemento?
5. ¿Cuál es el resultado de multiplicar cualquier matriz `A` por la matriz identidad `I` (asumiendo dimensiones compatibles)?

## 🚀 Próximos Pasos

¡Excelente trabajo! Has aprendido el 'cómo' y el 'porqué' computacional de las operaciones matriciales, y has visualizado su poder para transformar datos.

- En el siguiente notebook, **`1.1.2.3_Transformaciones_Lineales`**, formalizaremos las ideas que hemos explorado aquí, definiendo rigurosamente qué es una transformación lineal y explorando sus propiedades fundamentales como el Kernel y la Imagen, que son cruciales para entender conceptos como la compresión de datos con PCA.